This notebook computes the walking times between pairs of voting locations. While this notebook specifically does Los Angeles County, one can similarly do any other region by replacing certain values (marked with !! comments) with desired ones. 

In [1]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
from shapely.geometry import MultiPoint
import networkx as nx
import osmnx as ox

Example: Los Angeles County

load data

In [2]:
#loading street network
place = 'Los Angeles County' #!! replace with desired city here
#G = ox.graph_from_place(place,network_type='walk', buffer_dist = 5000, simplify=False)
G = ox.graph_from_place(place,network_type='walk', simplify=False)

In [3]:
from pyogrio import list_drivers
pyogrio.__gdal_version__

ImportError: dlopen(/Users/andreakbarreiro/miniconda3/envs/TDAEnv/lib/python3.13/site-packages/pyogrio/_vsi.cpython-313-darwin.so, 0x0002): Library not loaded: @rpath/libnetcdf.19.dylib
  Referenced from: <A2860E87-DCA6-3E1C-869C-97EF1B16E899> /Users/andreakbarreiro/miniconda3/envs/TDAEnv/lib/libgdal.32.3.6.2.dylib
  Reason: tried: '/Users/andreakbarreiro/miniconda3/envs/TDAEnv/lib/libnetcdf.19.dylib' (no such file), '/Users/andreakbarreiro/miniconda3/envs/TDAEnv/lib/python3.13/site-packages/pyogrio/../../../libnetcdf.19.dylib' (no such file), '/Users/andreakbarreiro/miniconda3/envs/TDAEnv/lib/python3.13/site-packages/pyogrio/../../../libnetcdf.19.dylib' (no such file), '/Users/andreakbarreiro/miniconda3/envs/TDAEnv/bin/../lib/libnetcdf.19.dylib' (no such file), '/Users/andreakbarreiro/miniconda3/envs/TDAEnv/bin/../lib/libnetcdf.19.dylib' (no such file), '/usr/local/lib/libnetcdf.19.dylib' (no such file), '/usr/lib/libnetcdf.19.dylib' (no such file, not in dyld cache)

In [6]:
import pandas as pd
#loading polling-site location
polls = pd.read_json('Los Angeles/lac_NoCatalina_polls.geojson') #!! specify the path/directory of polls dataset


nodes=ox.distance.nearest_nodes(G,polls.longitude,polls.latitude)
walk=np.full((len(lac_polls),len(lac_polls)),0)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [11]:
polls

NameError: name 'polls' is not defined

In [10]:
import json
polls = json.loads('Los Angeles/lac_NoCatalina_polls.geojson') 

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

multiprocess to speed up computation

In [ ]:
#get pairs to prepare for multiprocess
pairs_list = []
for i in range(len(nodes)):
    for j in range(i+1,len(nodes)):
        pairs_list.append((i,j))

In [ ]:
#define function needed for mp
def walk_dist_compute(pair):
    return nx.shortest_path_length(G,nodes[pair[0]],nodes[pair[1]],weight='length')/1.42

In [ ]:
import multiprocess as mp

pool = mp.Pool(60)
distances_list = pool.map(walk_dist_compute, pairs_list)
pool.close()

In [ ]:
#reassembling matrix
for i in range(len(pairs_list)):
    walk[pairs_list[i][0],pairs_list[i][1]]=distances_list[i]
    walk[pairs_list[i][1],pairs_list[i][0]]=distances_list[i]

In [ ]:
np.save('lac_walk.npy',walk) #name file as desired